In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install -q transformers biopython kaggle
print("✅ Đã cài đặt xong thư viện!")

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 30.1 MB/s eta 0:00:00
✅ Đã cài đặt xong thư viện!


In [2]:
import os
from google.colab import files

# Upload file kaggle.json
print("Vui lòng upload file kaggle.json của bạn:")
files.upload()

# Cấu hình Kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Tải dữ liệu cuộc thi (Sẽ mất khoảng 1-2 phút)
print("⏳ Đang tải dữ liệu CAFA 6...")
!kaggle competitions download -c cafa-6-protein-function-prediction
!unzip -q cafa-6-protein-function-prediction.zip -d /content/cafa6_data
print("✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data")

Vui lòng upload file kaggle.json của bạn:


Saving kaggle.json to kaggle.json
⏳ Đang tải dữ liệu CAFA 6...
  0% 0.00/91.3M [00:00<?, ?B/s]
100% 91.3M/91.3M [00:00<00:00, 1.43GB/s]
✅ Đã tải và giải nén dữ liệu tại /content/cafa6_data


In [4]:
import pandas as pd
import numpy as np

# ==============================================================================
# CẤU HÌNH ĐƯỜNG DẪN
# ==============================================================================
FILE_1 = '/content/IA.tsv'              # File cũ (hoặc mới tùy bạn đặt)
FILE_2 = '/content/cafa6_data/IA.tsv'   # File còn lại

print("⏳ Đang đọc file...")

# Giả định file IA.tsv không có header và có 2 cột: [term, ia]
# Nếu file có header, bạn bỏ tham số 'header=None' đi
try:
    df1 = pd.read_csv(FILE_1, sep='\t', header=None, names=['term', 'ia'])
    df2 = pd.read_csv(FILE_2, sep='\t', header=None, names=['term', 'ia'])
except Exception as e:
    print(f"❌ Lỗi đọc file: {e}")
    # Thử đọc có header nếu lỗi
    df1 = pd.read_csv(FILE_1, sep='\t')
    df2 = pd.read_csv(FILE_2, sep='\t')

print(f"✅ Đã tải xong.")
print(f"   - File 1 ({FILE_1}): {len(df1):,} dòng")
print(f"   - File 2 ({FILE_2}): {len(df2):,} dòng")

# ==============================================================================
# 1. SO SÁNH DANH SÁCH NHÃN (TERMS)
# ==============================================================================
set1 = set(df1['term'])
set2 = set(df2['term'])

only_in_1 = set1 - set2
only_in_2 = set2 - set1
common = set1 & set2

print("\n" + "="*50)
print("🔍 1. SO SÁNH DANH SÁCH NHÃN (TERMS)")
print("="*50)
print(f"   - Số nhãn chung: {len(common):,}")

if len(only_in_1) > 0:
    print(f"   ⚠️ Có {len(only_in_1)} nhãn chỉ có trong File 1.")
    print(f"      Ví dụ: {list(only_in_1)[:5]}")
else:
    print("   ✅ File 1 không thừa nhãn nào so với File 2.")

if len(only_in_2) > 0:
    print(f"   ⚠️ Có {len(only_in_2)} nhãn chỉ có trong File 2.")
    print(f"      Ví dụ: {list(only_in_2)[:5]}")
else:
    print("   ✅ File 2 không thừa nhãn nào so với File 1.")

# ==============================================================================
# 2. SO SÁNH GIÁ TRỊ IA (VALUES)
# ==============================================================================
print("\n" + "="*50)
print("🔍 2. SO SÁNH GIÁ TRỊ IA (VALUES)")
print("="*50)

# Merge 2 dataframe để so sánh dòng đối dòng
# Suffix _1 là file 1, _2 là file 2
merged = pd.merge(df1, df2, on='term', suffixes=('_1', '_2'), how='inner')

# Tìm những dòng có giá trị khác nhau (sai số > 1e-6)
# Lưu ý: Cần xử lý NaN nếu có
diff_mask = np.abs(merged['ia_1'] - merged['ia_2']) > 1e-6
diff_rows = merged[diff_mask]

if len(diff_rows) == 0:
    print("✅ Tuyệt vời! Tất cả các nhãn chung đều có giá trị IA giống hệt nhau.")
else:
    print(f"❌ PHÁT HIỆN SỰ KHÁC BIỆT!")
    print(f"   Có {len(diff_rows):,} nhãn có giá trị IA khác nhau.")
    print("-" * 60)
    print(f"{'Term':<15} | {'IA (File 1)':<15} | {'IA (File 2)':<15} | {'Diff'}")
    print("-" * 60)

    # In 10 dòng khác biệt đầu tiên
    for _, row in diff_rows.head(10).iterrows():
        print(f"{row['term']:<15} | {row['ia_1']:<15.4f} | {row['ia_2']:<15.4f} | {abs(row['ia_1']-row['ia_2']):.4f}")

    # Kiểm tra đặc biệt vụ lỗi 0.0
    zeros_1 = diff_rows[diff_rows['ia_1'] == 0.0]
    zeros_2 = diff_rows[diff_rows['ia_2'] == 0.0]

    if len(zeros_1) > 0:
        print(f"\n⚠️ Cảnh báo: File 1 có {len(zeros_1)} nhãn bị lỗi 0.0 (trong khi File 2 có giá trị).")
    if len(zeros_2) > 0:
        print(f"\n⚠️ Cảnh báo: File 2 có {len(zeros_2)} nhãn bị lỗi 0.0 (trong khi File 1 có giá trị).")

print("\n✅ Hoàn tất so sánh.")

⏳ Đang đọc file...
✅ Đã tải xong.
   - File 1 (/content/IA.tsv): 40,122 dòng
   - File 2 (/content/cafa6_data/IA.tsv): 40,122 dòng

🔍 1. SO SÁNH DANH SÁCH NHÃN (TERMS)
   - Số nhãn chung: 40,122
   ✅ File 1 không thừa nhãn nào so với File 2.
   ✅ File 2 không thừa nhãn nào so với File 1.

🔍 2. SO SÁNH GIÁ TRỊ IA (VALUES)
❌ PHÁT HIỆN SỰ KHÁC BIỆT!
   Có 6,216 nhãn có giá trị IA khác nhau.
------------------------------------------------------------
Term            | IA (File 1)     | IA (File 2)     | Diff
------------------------------------------------------------
GO:0000424      | 0.0000          | 4.3923          | 4.3923
GO:0000449      | 0.0000          | 9.0553          | 9.0553
GO:0000450      | 0.0000          | 9.0553          | 9.0553
GO:0000471      | 0.0000          | 6.3399          | 6.3399
GO:0000473      | 0.0000          | 6.2288          | 6.2288
GO:0000474      | 0.0000          | 7.2384          | 7.2384
GO:0000475      | 0.0000          | 9.0553          | 9.0553
G